In [1]:
# Gerekli Kütüphaneler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
import gc

## 1. Veriyi Yükle ve Eksiği Kaldır

In [2]:
print("Loading data...")
df = pd.read_csv('../pubg_predictor/data/train_V2.csv')
# Temizleme: 'winPlacePerc' sütununda eksik değerleri kaldırdık
df = df.dropna(subset=['winPlacePerc'])
print("Data loaded and successfully cleaned.")

Loading data...
Data loaded and successfully cleaned.


## 2. Feature Engineering

In [6]:
print("Starting feature engineering...")

## 1. Total Distance
df['totalDistance'] = df['rideDistance'] + df['walkDistance'] + df['swimDistance']

## 2. Health Items
df['healthItems'] = df['heals'] + df['boosts']

## 3. Headshot Rate
df['headshotRate'] = df['headshotKills'] / df['kills']
df['headshotRate'] = df['headshotRate'].fillna(0)

## 4. Teamwork
df['teamwork'] = df['assists'] + df['revives']

df = df.drop(columns=["Id", "groupId", "matchId"])
df = pd.get_dummies(df, columns=['matchType'])

print(f"Feature engineering completed! New shape: {df.shape}")

Starting feature engineering...
Feature engineering completed! New shape: (4446965, 45)


## 3. Model Eğitimi

In [7]:
# MODEL EĞİTİMİ
X = df.drop(columns=['winPlacePerc'])
y = df['winPlacePerc']

# RAM temizliği
del df
gc.collect()

# Train Test Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
print("Training with new features...")
model = LGBMRegressor(n_estimators=100, learning_rate=0.05, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

# Tahmin ve Skor
preds = model.predict(X_val)
mae = mean_absolute_error(y_val, preds)

print("-" * 40)
print(f"📉 NEW MAE SCORE: {mae:.5f}")
print(f"Old Score (Baseline): 0.06367")
print("-" * 40)

Training with new features...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.098397 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3208
[LightGBM] [Info] Number of data points in the train set: 3557572, number of used features: 44
[LightGBM] [Info] Start training from score 0.472937
----------------------------------------
📉 NEW MAE SCORE: 0.06337
Old Score (Baseline): 0.06367
----------------------------------------


## 4. Analiz
Görüldüğü üzere yaptığımız kolaylaştırmalar sonucunda 0.0003'lük bir iyileşme gerçekleşti. LightGBM yeteri kadar zeki bir model olduğu için pek etki etmedi o nedenle başka yöntemler deneyeceğiz.

## 5. Advanced Feature Engineering

In [ ]:
# Gereken Kütüphaneler
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor
import gc


print("Loading data...")
df = pd.read_csv('../pubg_predictor/data/train_V2.csv')
# Temizleme: 'winPlacePerc' sütununda eksik değerleri kaldırdık
df = df.dropna(subset=['winPlacePerc'])
print("Data loaded and successfully cleaned.")

# --- FEATURE ENGINEERING 1: BASİT TÜRETMELER ---
print("Basic Features are being created...")
df['totalDistance'] = df['walkDistance'] + df['rideDistance'] + df['swimDistance']
df['healthItems'] = df['heals'] + df['boosts']
df['headshotRate'] = df['headshotKills'] / df['kills']
df['headshotRate'] = df['headshotRate'].fillna(0)
df['teamwork'] = df['revives'] + df['assists']

# --- FEATURE ENGINEERING 2: GRUPLAMALAR ---
print("Advanced Features are being created. This may take a while...")

# Takım (GroupId) Büyüklüğü: Takımda kaç kişi var?
# (Bazı modlarda takım arkadaşı sayısı avantaj sağlar)
df['groupSize'] = df.groupby('groupId')['groupId'].transform('count')

# Maçtaki Oyuncu Sayısı: (100 kişilik maç ile 80 kişilik maç farklıdır)
df['matchSize'] = df.groupby('matchId')['matchId'].transform('count')

# Oyuncunun başarısını maçın ortalamasıyla kıyasla her maçın kendi içindeki 'kills' ve 'damageDealt' ortalamasını buluyoruz
df['matchMeanKills'] = df.groupby('matchId')['kills'].transform('mean')
df['matchMeanDamage'] = df.groupby('matchId')['damageDealt'].transform('mean')

# Oyuncunun performansı / Maç ortalaması (Rank normalization)
df['killsRel'] = df['kills'] / df['matchMeanKills']
df['damageRel'] = df['damageDealt'] / df['matchMeanDamage']
df['killsRel'] = df['killsRel'].fillna(0)
df['damageRel'] = df['damageRel'].fillna(0)

# Takım içi maksimum performans (Takımın en iyisi kim?)
df['maxTeamKills'] = df.groupby('groupId')['kills'].transform('max')
df['maxTeamDamage'] = df.groupby('groupId')['damageDealt'].transform('max')

# -------------------------------------------------------

# ID'lere ihtiyacımız yok
df = df.drop(columns=['Id', 'groupId', 'matchId'])

# Kategorik Dönüşüm
df = pd.get_dummies(df, columns=['matchType'])

# MODEL EĞİTİMİ
print("Model is being trained with new features...")
X = df.drop(columns=['winPlacePerc'])
y = df['winPlacePerc']

# RAM Temizliği
del df
gc.collect()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = LGBMRegressor(n_estimators=100, learning_rate=0.05, n_jobs=-1, random_state=42)
model.fit(X_train, y_train)

preds = model.predict(X_val)
mae = mean_absolute_error(y_val, preds)

print("-" * 40)
print(f"🚀 NEW MAE SCORE: {mae:.5f}")
print("Old Score: 0.06337")
print("-" * 40)

Loading data...
Data loaded and successfully cleaned.
Basic Features are being created...
Advanced Features are being created. This may take a while...
Model is being trained with new features...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.113496 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4645
[LightGBM] [Info] Number of data points in the train set: 3557572, number of used features: 52
[LightGBM] [Info] Start training from score 0.472937
----------------------------------------
🚀 NEW MAE SCORE: 0.05892
Old Score: 0.06337
----------------------------------------


## 6. Analiz-2
Ham verilerle eğitilen baseline modelin hatası 0.0636 iken, 'maç içi ortalama performans' (relative performance) ve 'grup istatistikleri' eklendiğinde hata 0.0589'a düştü. Bu durum, bir oyuncunun mutlak istatistiklerinden ziyade, rakiplerine göre ne kadar iyi oynadığının sıralamayı belirlediğini kanıtlamaktadır. Yapmış olduğumuz yeni geliştirmeler işe yaradı 0.06337 hata payından, 0.05892'ye düştü.